In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from multiprocessing import *
import warnings
warnings.filterwarnings("ignore")


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.metrics import roc_auc_score

# Regularized Greedy Forest
from rgf.sklearn import RGFClassifier     # https://github.com/fukatani/rgf_python
from catboost import CatBoostClassifier

from numba import jit
import time
import gc
import subprocess
import glob

import time
start_time = time.time()
tcurrent   = start_time

np.random.seed(31163)  # v04

In [2]:
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [3]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,    # Revised to encode validation series
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [4]:
# Read data
train_df = pd.read_csv('/Users/siero5335/Desktop/Safe Driver Prediction/train.csv', na_values="-1") # .iloc[0:200,:]
test_df = pd.read_csv('/Users/siero5335/Desktop/Safe Driver Prediction/test.csv', na_values="-1")

In [5]:
train_df['v001'] = train_df["ps_ind_03"]+train_df["ps_ind_14"]+np.square(train_df["ps_ind_15"])
train_df['v002'] = train_df["ps_ind_03"]+train_df["ps_ind_14"]+np.tanh(train_df["ps_ind_15"])
train_df['v003'] = train_df["ps_reg_01"]+train_df["ps_reg_02"]**3+train_df["ps_reg_03"]
train_df['v004'] = train_df["ps_reg_01"]**2.15+np.tanh(train_df["ps_reg_02"])+train_df["ps_reg_03"]**3.1
train_df['v005'] = train_df["ps_calc_01"]+train_df["ps_calc_13"]+np.tanh(train_df["ps_calc_14"])
train_df['v006'] = train_df["ps_car_13"]+np.tanh(train_df["v003"])
train_df['v007'] = train_df["ps_car_13"]+train_df["v002"]**2.7
train_df['v008'] = train_df["ps_car_13"]+train_df["v003"]**3.4
train_df['v009'] = train_df["ps_car_13"]+train_df["v004"]**3.1
train_df['v010'] = train_df["ps_car_13"]+train_df["v005"]**2.3

train_df['v011'] = train_df["ps_ind_03"]**2.1+train_df["ps_ind_14"]**0.45+train_df["ps_ind_15"]**2.4
train_df['v012'] = train_df["ps_ind_03"]**2.56+train_df["ps_calc_13"]**2.15+train_df["ps_reg_01"]**2.3
train_df['v013'] = train_df["v003"]**2.15+train_df["ps_reg_01"]**2.49+train_df["ps_ind_15"]**2.14
train_df['v014'] = train_df["v009"]**2.36+train_df["ps_calc_01"]**2.25+train_df["ps_reg_01"]**2.36
train_df['v015'] = train_df["v003"]**3.21+0.001*np.tanh(train_df["ps_reg_01"])+train_df["ps_ind_15"]**3.12
train_df['v016'] = train_df["v009"]**2.13+0.001*np.tanh(train_df["ps_calc_01"])+train_df["ps_reg_01"]**2.13
train_df['v017'] = train_df["v016"]**2+train_df["v001"]**2.1+train_df["v003"]**2.3

# test
test_df['v001'] = test_df["ps_ind_03"]+test_df["ps_ind_14"]+np.square(test_df["ps_ind_15"])
test_df['v002'] = test_df["ps_ind_03"]+test_df["ps_ind_14"]+np.tanh(test_df["ps_ind_15"])
test_df['v003'] = test_df["ps_reg_01"]+test_df["ps_reg_02"]**3+test_df["ps_reg_03"]
test_df['v004'] = test_df["ps_reg_01"]**2.15+np.tanh(test_df["ps_reg_02"])+test_df["ps_reg_03"]**3.1
test_df['v005'] = test_df["ps_calc_01"]+test_df["ps_calc_13"]+np.tanh(test_df["ps_calc_14"])
test_df['v006'] = test_df["ps_car_13"]+np.tanh(test_df["v003"])
test_df['v007'] = test_df["ps_car_13"]+test_df["v002"]**2.7
test_df['v008'] = test_df["ps_car_13"]+test_df["v003"]**3.4
test_df['v009'] = test_df["ps_car_13"]+test_df["v004"]**3.1
test_df['v010'] = test_df["ps_car_13"]+test_df["v005"]**2.3

test_df['v011'] = test_df["ps_ind_03"]**2.1+test_df["ps_ind_14"]**0.45+test_df["ps_ind_15"]**2.4
test_df['v012'] = test_df["ps_ind_03"]**2.56+test_df["ps_calc_13"]**2.15+test_df["ps_reg_01"]**2.3
test_df['v013'] = test_df["v003"]**2.15+test_df["ps_reg_01"]**2.49+test_df["ps_ind_15"]**2.14
test_df['v014'] = test_df["v009"]**2.36+test_df["ps_calc_01"]**2.25+test_df["ps_reg_01"]**2.36
test_df['v015'] = test_df["v003"]**3.21+0.001*np.tanh(test_df["ps_reg_01"])+test_df["ps_ind_15"]**3.12
test_df['v016'] = test_df["v009"]**2.13+0.001*np.tanh(test_df["ps_calc_01"])+test_df["ps_reg_01"]**2.13
test_df['v017'] = test_df["v016"]**2+test_df["v001"]**2.1+test_df["v003"]**2.3

In [6]:
# from olivier
# from olivier
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
	
	"v001","v002","v003","v004","v005",
	"v006","v007","v008","v009","v010",   
	"v011","v012","v013","v014","v015",   
	"v016","v017" # new nonlinear features
]

# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [7]:
# Process data
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

In [8]:
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]


y_valid_pred = 0*y
y_test_pred = 0

In [9]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2017).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
                cross_score = cross_val_score(clf, X_train, y_train, cv=self.n_splits, scoring='roc_auc')
                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

            print("     Model score: %.5f\n" % roc_auc_score(y, S_train[:,i]))

        self.base_preds = S_test
        
        # Log odds transformation
        almost_zero = 1e-12
        almost_one = 1 - almost_zero  # To avoid division by zero
        S_train[S_train>almost_one] = almost_one
        S_train[S_train<almost_zero] = almost_zero
        S_train = np.log(S_train/(1-S_train))
        S_test[S_test>almost_one] = almost_one
        S_test[S_test<almost_zero] = almost_zero
        S_test = np.log(S_test/(1-S_test))
        
        results = cross_val_score(self.stacker, S_train, y, cv=self.n_splits, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        print( 'Coefficients:', self.stacker.coef_ )

        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [10]:
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['random_state'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['random_state'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['random_state'] = 99


# XGBoost params
xgb_params1 = {}
xgb_params1['objective'] = 'binary:logistic'
xgb_params1['learning_rate'] = 0.02
xgb_params1['n_estimators'] = 1100
xgb_params1['max_depth'] = 4
xgb_params1['subsample'] = 0.8
xgb_params1['colsample_bytree'] = 0.8 
xgb_params1['min_child_weight'] = 2.4073000000000002
xgb_params1['reg_alpha'] = 8.0701999999999998
xgb_params1['seed'] = 71
xgb_params1['gamma'] = 0.15110000000000001
xgb_params1['reg_lambda'] =  2.0125999999999999
xgb_params1['scale_pos_weight'] =  2.2281


# Regularized Greedy Forest params
rgf_params = {}
rgf_params['max_leaf'] = 1000
rgf_params['learning_rate'] = 0.3
rgf_params['algorithm'] = "RGF"
rgf_params['test_interval'] = 100
rgf_params['min_samples_leaf'] = 10 
rgf_params['l2'] = 0.01  
rgf_params['sl2'] = 0.01

#CatBoost params
cat_params = {}
cat_params['iterations'] = 900
cat_params['depth'] = 4
cat_params['rsm'] = 0.95
cat_params['learning_rate'] = 0.03
cat_params['l2_leaf_reg'] = 3.5  
cat_params['border_count'] = 8
cat_params['gradient_iterations'] = 4

In [13]:
cat_model = CatBoostClassifier(**cat_params)

lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

xgb_model1 = XGBClassifier(**xgb_params1)

rgf_model = RGFClassifier(**rgf_params)

log_model = LogisticRegression(fit_intercept=True)

In [14]:
log_model = LogisticRegression(fit_intercept=True)

stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (cat_model, lgb_model, lgb_model2, lgb_model3, xgb_model1))        
        
y_pred = stack.fit_predict(X, y, test_df)        

Fit <catboost.core.CatBoostClassifier object at 0x1a1d42ada0> fold 1
    cross_score: 0.63903
Fit <catboost.core.CatBoostClassifier object at 0x1a1d42ada0> fold 2
    cross_score: 0.63730
Fit <catboost.core.CatBoostClassifier object at 0x1a1d42ada0> fold 3
    cross_score: 0.63688
     Model score: 0.63893

Fit LGBMClassifier fold 1
    cross_score: 0.63866
Fit LGBMClassifier fold 2
    cross_score: 0.63606
Fit LGBMClassifier fold 3
    cross_score: 0.63732
     Model score: 0.64111

Fit LGBMClassifier fold 1
    cross_score: 0.63617
Fit LGBMClassifier fold 2
    cross_score: 0.63579
Fit LGBMClassifier fold 3
    cross_score: 0.63689
     Model score: 0.64023

Fit LGBMClassifier fold 1
    cross_score: 0.63468
Fit LGBMClassifier fold 2
    cross_score: 0.63376
Fit LGBMClassifier fold 3
    cross_score: 0.63462
     Model score: 0.63831

Fit XGBClassifier fold 1
    cross_score: 0.63709
Fit XGBClassifier fold 2
    cross_score: 0.63588
Fit XGBClassifier fold 3
    cross_score: 0.63743
 

In [15]:
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred
sub.to_csv('stacked_16_lgb_xgb_cat.csv', index=False)